# *Resultados*
* * nan -> median
* categoric(onehotcode)+Number
* R²: 0.9150 |0.9093(log)
* ⏱️ Tempo de execução: 4.43 segundos

## 1. Libraries

In [1]:
# =====================================================
# 🏠 House Prices - Pipelines
# =====================================================
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning, 
                       message='Found unknown categories in columns')
import time
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

# Scikit-learn - Model selection e avaliação
#from sklearn.model_selection import train_test_split, cross_val_score, KFold, RandomizedSearchCV

# Scikit-learn - Pré-processamento e pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Scikit-learn - Modelos lineares
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV

# Scikit-learn - Ensemble methods
from sklearn.ensemble import RandomForestRegressor

# Scikit-learn - Métricas de avaliação
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# XGBoost
from xgboost import XGBRegressor

# Distribuições para busca de hiperparâmetros
from scipy.stats import randint, uniform,loguniform

from setup_notebook import setup_path
setup_path()
from src.model_utils import *


In [2]:
# =====================================================
# 📁 1. Leitura dos dados
# =====================================================
dfo = pd.read_csv("/home/akel/PycharmProjects/Kaggle/HousePrices/data/train.csv")

## 2. Models

In [3]:
df_train=dfo.copy()
# =====================================================
# 🧹 2. Pré-processamento inicial
# =====================================================
# remoção de colunas com muitos nulos (> 10%)
colnull_train=df_train.columns[(df_train.isnull().sum()/df_train.shape[0]>0.1)] # 
df_train=df_train.drop(columns=colnull_train,axis=1)

id_train=df_train['Id']

# obtendo nome das variáveis categóricas e numéricas
num_features = df_train.select_dtypes(include=['number']).columns.drop(['Id', 'SalePrice'])
cat_features = df_train.select_dtypes(include=['object']).columns

# =====================================================
# 🧩 3. Pré-processadores
# =====================================================
# NAN -> median
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# categoric -> binario onehotcode 
cat_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first',
                             sparse_output=False,
                             handle_unknown='ignore'))
])

preprocessador = ColumnTransformer(transformers=[
    ('cat', cat_transformer, cat_features),
    ('num', num_transformer, num_features)   
],verbose_feature_names_out=False) 


# =====================================================
#  🤖 4.Modelos
# =====================================================

model_lr=LinearRegression()
model_LR=Pipeline([ ('preprocess', preprocessador), 
                        ('model', model_lr )])

model_rf1 = RandomForestRegressor(random_state=42)
model_RF1 = Pipeline([ ('preprocess', preprocessador), 
                        ('model',model_rf1 )])

model_rf2=RandomForestRegressor(bootstrap=True, max_depth=25, max_features=0.3,
                      min_samples_split=2,  min_samples_leaf= 1,n_estimators=100, n_jobs=-1,
                      random_state=42)
model_RF2 = Pipeline([ ('preprocess', preprocessador), 
                        ('model',model_rf2 )])

model_xg1 = XGBRegressor(objective='reg:squarederror', random_state=42)
model_XGB1 = Pipeline([ ('preprocess', preprocessador), 
                        ('model', model_xg1 )])

model_xg2 = XGBRegressor( objective='reg:squarederror', n_estimators=700,
                         subsample= 0.6,reg_lambda= 0.5,reg_alpha= 1.0,
                         max_depth= 3,learning_rate= 0.073,
                         colsample_bytree= 0.7,random_state=42,
                         n_jobs=-1 ) 
model_XGB2 = Pipeline([ ('preprocess', preprocessador), 
                        ('model',model_xg2 )])

## 3. Run models

In [4]:
X=df_train.drop(['Id', 'SalePrice'], axis=1)
y=np.log1p(df_train['SalePrice'])
#y=df_train['SalePrice']
start_time = time.time()
r2=run_model(X,y,model_LR,'LR')

r2=run_model(X,y,model_RF1,'RF1')

r2=run_model(X,y,model_RF2,'RF2')

run_model(X,y,model_XGB1,'XGB1')  
run_model(X,y,model_XGB2,'XGB2')



Modelo: 🤖 LR
MAE: 0.095
RMSE: 0.1826
R²: 0.8035
Modelo: 🤖 RF1
MAE: 0.0934
RMSE: 0.1384
R²: 0.8871
Modelo: 🤖 RF2
MAE: 0.0905
RMSE: 0.1375
R²: 0.8886
Modelo: 🤖 XGB1
MAE: 0.0976
RMSE: 0.145
R²: 0.876
Modelo: 🤖 XGB2
MAE: 0.0865
RMSE: 0.1255
R²: 0.9071


0.9071005180824829

In [5]:
# #===========================================================
# cv = KFold(n_splits=5, shuffle=True, random_state=42)

# param_dist_rf = {
#     'n_estimators': np.arange(100, 800, 100),  # número de árvores
#     'max_depth': [None, 5, 10, 15, 20, 25],      # profundidade máxima
#     'min_samples_split': [2, 5, 10, 15],         # mínimo de amostras para dividir um nó
#     'min_samples_leaf': [1, 2, 4, 6],            # mínimo de amostras por folha
#     'max_features': [0.3, 0.5, 0.7, 0.8, 1.0, 'sqrt',None],       # nº de features usadas em cada split
#     'bootstrap': [True]                   # amostragem com ou sem reposição
# }
# rf=RandomForestRegressor(random_state=42, n_jobs=-1)
# rand_rf = RandomizedSearchCV(
#     estimator=rf,
#     param_distributions=param_dist_rf,
#     n_iter=50,                   
#     cv=cv,
#     scoring='r2',
#     random_state=42,
#     n_jobs=-1,
#     verbose=2,
# error_score='raise')

# model_RSearch_rf = Pipeline([ ('preprocess', preprocessador), 
#                         ('model',rand_rf )])

# param_dist_xg = {
#     'n_estimators': np.arange(200, 1000, 100),
#     'learning_rate': np.linspace(0.01, 0.2, 10),
#     'max_depth': np.arange(3, 8),
#     'subsample': np.linspace(0.6, 1.0, 5),
#     'colsample_bytree': np.linspace(0.6, 1.0, 5),
#     'reg_alpha': np.linspace(0, 1, 5),
#     'reg_lambda': np.linspace(0.5, 2, 5)}

# xb_s = XGBRegressor(
#         objective='reg:squarederror',
#         eval_metric='rmse',
#         random_state=42,
#         tree_method='hist',  # acelera em CPU
#         n_jobs=-1)

# rand_xgb = RandomizedSearchCV(
#     estimator=xb_s,
#     param_distributions=param_dist_xg,
#     n_iter=50,  # número de combinações testadas
#     cv=cv,
#     scoring='r2',
#     random_state=42,
#     n_jobs=-1,
#     verbose=2
# )

# model_RSearch_xgb = Pipeline([ ('preprocess', preprocessador), 
#                          ('model',rand_xgb)])

In [6]:
# X=df_train.drop(['Id', 'SalePrice'], axis=1)
# y=np.log1p(df_train['SalePrice'])
# start_time = time.time()

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# model_RSearch_rf.fit(X_train,y_train)
# end_time = time.time()
# print(f"⏱️ Tempo de execução: {end_time - start_time:.2f} segundos")
# rand_rf.best_params_